In [ ]:
import psycopg2
import numpy as np
import pandas as pd
from sklearn.preprocessing import MaxAbsScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Conv1D, SpatialDropout1D, LSTM, Dense, Dropout, LayerNormalization, Input
from tensorflow.keras.layers import MultiHeadAttention
from tensorflow.keras.regularizers import l2, l1_l2,l1
import keras_tuner as kt
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import Callback

# Conecta a la base de datos PostgreSQL y recupera los datos históricos de precios de una criptomoneda específica. 
# Devuelve un DataFrame con los datos ordenados por fecha.
def fetch_data(ticker, conn_params):
    conn = psycopg2.connect(**conn_params)
    query = f"""
    SELECT * FROM crypto_prices
    WHERE ticker = '{ticker}'
    ORDER BY date;
    """
    df = pd.read_sql_query(query, conn)
    conn.close()
    return df

#  Convierte la columna de fecha en un índice de tiempo, selecciona características relevantes
#  y las escala usando MaxAbsScaler para normalizar los valores.
def preprocess_data(df):
    df['date'] = pd.to_datetime(df['date'])
    df.set_index('date', inplace=True)
    features = df[['caracteristicas']]
    
    scaler = MaxAbsScaler()
    scaled_features = scaler.fit_transform(features)
    return scaled_features
    
# Genera conjuntos de entrenamiento con una ventana temporal definida, time_step, para predecir 
# el precio de cierre (columna 3) basado en los datos anteriores.
def create_dataset(data, time_step=12):
    X, y = [], []
    for i in range(len(data) - time_step - 1):
        X.append(data[i:(i + time_step), :])
        y.append(data[i + time_step, 3])  # Asumiendo que la columna 3 es el precio de cierre
    return np.array(X), np.array(y)

# Calcula la métrica sMAPE (Symmetric Mean Absolute Percentage Error)
#  para evaluar el error porcentual medio entre valores predichos y reales.
def smape(y_true, y_pred):
    numerator = tf.abs(y_true - y_pred)
    denominator = (tf.abs(y_true) + tf.abs(y_pred)) / 2
    smape_value = tf.where(tf.equal(denominator, 0), tf.zeros_like(numerator), numerator / denominator)
    return tf.reduce_mean(smape_value) * 100  # Convertir a porcentaje

# Define métrica de perdida MAE
def mean_absolute_error_tf(y_true, y_pred):
    return tf.reduce_mean(tf.abs(y_true - y_pred))


# Callback personalizado que reduce el valor de la regularización L2 en las capas específicas 
# al final de cada época, ajustando la penalización L2 para evitar el sobreajuste
class L2DecayCallback(Callback):
    def __init__(self, initial_l2, decay_rate, target_layers):
        super().__init__()
        self.initial_l2 = initial_l2
        self.decay_rate = decay_rate
        self.target_layers = target_layers

    def on_epoch_end(self, epoch, logs=None):
        current_l2 = self.initial_l2 * (self.decay_rate ** epoch)
        for layer in self.target_layers:
            if hasattr(layer, 'kernel_regularizer'):
                layer.kernel_regularizer.l2 = current_l2

# Construye un modelo secuencial con capas Conv1D, LSTM y una capa de atención multi-cabezal, 
# con regularización L2 que decae durante el entrenamiento.
# Se utiliza Spatial Dropout para reducir la posibilidad de sobre ajuste y l1 en capa de atención multi-cabezal
# para mejorar la selección de características
# Utiliza Keras Tuner para optimizar hiperparámetros como el número de filtros y la tasa de aprendizaje.
def build_model_with_decay(hp, X_train, initial_l2, decay_rate):
    # Definición de la entrada
    inputs = Input(shape=(X_train.shape[1], X_train.shape[2]))
    
    # Capa Conv1D con Spatial Dropout
    conv_layer = Conv1D(
        filters=hp.Int('filters', min_value=32, max_value=128, step=32),
        kernel_size=3,
        activation='relu',
        kernel_regularizer=l2(initial_l2)
    )(inputs)
    conv_layer = SpatialDropout1D(rate=0.2)(conv_layer)  # Añadir Spatial Dropout

    # Capa LSTM con Spatial Dropout
    lstm_layer = LSTM(
        units=hp.Int('units_layer_1', min_value=50, max_value=500, step=50),
        return_sequences=True,
        kernel_regularizer=l2(initial_l2)
    )(conv_layer)
    lstm_layer = SpatialDropout1D(rate=0.2)(lstm_layer)  # Añadir Spatial Dropout

    # Capa de Atención Multihead
    attention_layer = MultiHeadAttention(
        num_heads=hp.Int('num_heads', min_value=2, max_value=8, step=1),
        key_dim=hp.Int('key_dim', min_value=8, max_value=64, step=8),
        kernel_regularizer=l1(0.01)
    )(lstm_layer, lstm_layer)  # Aplicar atención sobre la salida de LSTM

    # Normalización después de la capa de atención
    attention_output = LayerNormalization()(attention_layer)

    # Capa de salida
    output = Dense(1, kernel_regularizer=l2(initial_l2))(attention_output)

    # Crear el modelo
    model = Model(inputs=inputs, outputs=output)

    # Compilación del modelo
    model.compile(optimizer=tf.keras.optimizers.SGD(
        learning_rate=hp.Float('learning_rate', min_value=1e-5, max_value=1e-2, sampling='LOG')
    ), loss='mean_squared_error')

    return model  # Solo retorna el modelo

# Almacena las métricas de error en una tabla de la base de datos, lo que permite analizar 
# y comparar el rendimiento de diferentes modelos.
def store_errors(ticker, mse, mae, r2, mape, smape_val, conn_params):
    mse = float(mse)
    mae = float(mae)
    r2 = float(r2)
    mape = float(mape)
    smape_val = float(smape_val)
    
    conn = psycopg2.connect(**conn_params)
    cursor = conn.cursor()
    
    query = """
    INSERT INTO error (ticker, mse, mae, r2, mape, smape, coment)
    VALUES (%s, %s, %s, %s, %s, %s, ' ');
    """
    cursor.execute(query, (ticker, mse, mae, r2, mape, smape_val))
    
    conn.commit()
    cursor.close()
    conn.close()

# Realiza el ciclo completo de entrenamiento y evaluación para cada criptomoneda en la lista de tickers.
#  Divide los datos en conjuntos de entrenamiento y validación, usa Keras Tuner para buscar la mejor 
# configuración de hiperparámetros utilizando busqueda bayesiana, evalúa el modelo con métricas de rendimiento y guarda el mejor 
# modelo en un archivo.
def train_and_evaluate(tickers, conn_params, time_step=48):
    for ticker in tickers:
        print(f"Processing ticker: {ticker}")

        df = fetch_data(ticker, conn_params)
        scaled_features = preprocess_data(df)
        X, y = create_dataset(scaled_features, time_step)
        
        split_idx = int(len(X) * 0.8)
        X_train, X_val = X[:split_idx], X[split_idx:]
        y_train, y_val = y[:split_idx], y[split_idx:]

        # Definir el tuner usando Bayesian Optimization
        tuner = kt.BayesianOptimization(
            lambda hp: build_model_with_decay(hp,
            X_train,
            0.01,
            decay_rate=0.9 
            ),
            objective='val_loss',
            max_trials=50,  # Número máximo de combinaciones a probar
            directory='my_dir',
            project_name=f'crypto_lstm_{ticker}'
        )

        # Realizar la búsqueda de hiperparámetros
        fixed_batch_size = 128
        tuner.search(X_train, y_train, epochs=150, validation_split=0.2, 
                     batch_size=fixed_batch_size,
                     callbacks=[EarlyStopping(monitor='val_loss', patience=3)])

        # Evaluar el mejor modelo
        best_model = tuner.get_best_models(num_models=1)[0]
        y_pred = best_model.predict(X_val)
        
        mse = mean_squared_error(y_val, y_pred)
        mae = mean_absolute_error(y_val, y_pred)
        r2 = r2_score(y_val, y_pred)
        mape = np.mean(np.abs((y_val - y_pred) / y_val)) * 100

        smape_val = smape(tf.convert_to_tensor(y_val, dtype=tf.float32), 
                          tf.convert_to_tensor(y_pred, dtype=tf.float32)).numpy()
        
        print(f"Ticker: {ticker}, MSE: {mse}, MAE: {mae}, R²: {r2}, MAPE: {mape}, sMAPE: {smape_val}")
        
        model_path = f"model_{ticker}.keras"
        best_model.save(model_path)
        print(f"Model saved to {model_path}")
        
        store_errors(ticker, mse, mae, r2, mape, smape_val, conn_params)

# Parametros de conexion a BDD
conn_params = {
    'dbname': 'dbname',
    'user': 'user',
    'password': 'password',
    'host': 'localhost',
    'port': 'port'
}
tickers = [
"PAXGUSDT"
]
train_and_evaluate(tickers, conn_params)


In [ ]:
import psycopg2
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import MaxAbsScaler

#Conecta a la base de datos PostgreSQL y recupera datos históricos de 
# precios de una criptomoneda específica, ordenados por fecha.
def fetch_data(ticker, conn_params):
    conn = psycopg2.connect(**conn_params)
    query = f"""
    SELECT * FROM crypto_prices
    WHERE ticker = '{ticker}'
    ORDER BY date;
    """
    df = pd.read_sql_query(query, conn)
    conn.close()
    return df

#Recupera el valor de timestep desde la tabla timestep en la base de datos para un ticker dado.
def fetch_timestep(ticker, conn_params):
    conn = psycopg2.connect(**conn_params)
    query = f"""
    SELECT timestep FROM timestep
    WHERE ticker = '{ticker}';
    """
    cursor = conn.cursor()
    cursor.execute(query)
    timestep = cursor.fetchone()[0]  # Obtener el primer resultado (timestep)
    conn.close()
    return int(timestep)  # Convertir el timestep a entero


# Preprocesa los datos, convirtiendo fechas, escalando características y 
# devolviendo el último precio real junto con el escalador utilizado.
def preprocess_data(df):
    df['date'] = pd.to_datetime(df['date'])
    df.set_index('date', inplace=True)
    features = df[['caracteristicas']]
    
    # Aplicar el escalado
    scaler = MaxAbsScaler()
    scaled_features = scaler.fit_transform(features)

    return scaled_features, features['close'].values[-1], scaler  # Retornar el último precio real y el scaler

# Carga un modelo previamente guardado en formato Keras, personalizado con las métricas sMAPE y MAE.
def load_model(ticker):
    model_path = f"modelo_{ticker}.keras"
    return tf.keras.models.load_model(model_path, custom_objects={"smape": smape, "mean_absolute_error_tf": mean_absolute_error_tf})

# Se define para cargar correctamente el modelo guardado con métricas personalizadas
def smape(y_true, y_pred):
    numerator = tf.abs(y_true - y_pred)
    denominator = (tf.abs(y_true) + tf.abs(y_pred)) / 2
    return tf.reduce_mean(numerator / denominator) * 100  # Convertir a porcentaje

# Se define para cargar correctamente el modelo guardado con métricas personalizadas
def mean_absolute_error_tf(y_true, y_pred):
    return tf.reduce_mean(tf.abs(y_true - y_pred))

# Calcula el porcentaje de ganancia o pérdida basado en los precios predichos y reales, evitando divisiones por cero.
def calculate_profit_percentage(predicted_price, actual_price):
    if actual_price != 0:
        return ((predicted_price - actual_price) / actual_price) * 100
    return 0  # Para evitar la división por cero

# Inserta el porcentaje de ganancia o pérdida en la tabla profit de la base de datos.
def store_profit(ticker, profit, conn_params):
    conn = psycopg2.connect(**conn_params)
    cursor = conn.cursor()
    
    query = """
    INSERT INTO profit (ticker, profit)
    VALUES (%s, %s);
    """
    cursor.execute(query, (ticker, profit))
    
    conn.commit()
    cursor.close()
    conn.close()

# Realiza el proceso de predicción para una lista de tickers, preprocesa datos, carga el modelo, predice 
# el próximo precio y calcula la ganancia o pérdida, almacenando los resultados en la base de datos.
def prediction_module(tickers, conn_params):
    for ticker in tickers:
        print(f"Processing ticker: {ticker}")

        # Obtener y preprocesar los datos
        df = fetch_data(ticker, conn_params)
        scaled_features, actual_price, scaler = preprocess_data(df)

        # Obtener el timestep desde la tabla 'time'
        time_step = fetch_timestep(ticker, conn_params)

        # Hacer predicciones para el próximo intervalo con el timestep obtenido
        if len(scaled_features) >= time_step:  # Asegurarse de tener suficientes datos para predecir
            last_data = scaled_features[-time_step:].reshape(1, time_step, scaled_features.shape[1])  # Usar los últimos timestep
        else:
            print(f"Not enough data to make a prediction for ticker: {ticker}")
            continue

        # Cargar el modelo guardado
        model = load_model(ticker)

        # Hacer la predicción
        predicted_price_scaled = model.predict(last_data)[0, 0]

        # Invertir el escalado del precio predicho
        predicted_price = scaler.inverse_transform(np.array([[predicted_price_scaled] + [0] * (scaled_features.shape[1] - 1)])[0].reshape(1, -1))[0][0]

        # Calcular la ganancia o pérdida
        profit_percentage = calculate_profit_percentage(predicted_price, actual_price)

        # Imprimir el resultado
        print(f"Ticker: {ticker}, Predicted Price: {predicted_price}, Actual Price: {actual_price}, Profit/Loss: {profit_percentage}%")

        # Almacenar la ganancia en la base de datos
        store_profit(ticker, profit_percentage, conn_params)

# Parámetros de conexión a BDD
conn_params = {
    'dbname': 'crypto8',
    'user': 'postgres',
    'password': 'loquesea',
    'host': 'localhost',
    'port': '5432'
}
tickers = [
"BTCSDT" #Bitcoin
]  # Lista de tickers

prediction_module(tickers, conn_params) #Ejecutar